# MATH 468 - Stochastic Calculus II - HW 6

In [1]:
'''
Packages Used
---
Data - pandas, datetime, yfinance
Numerical - numpy, scipy stats
Plotting - Matplotlib, seaborn
Custom - StoCalCodeDeck
'''
import pandas as pd
import datetime
import yfinance as yf

import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

import StoCalCodeDeck as stocal

## Problem 1 - Pricing and the Greeks

We consider a (european) call option on a stock with expiration in 3 months ($T = \frac{1}{4}$) and strike price $K=10$.  The annual interest rate on the market is $r=2\%$.  The current price of the stock is $S_0=10$ and we assume that the stock follows a geometric Brownian motion model with parameters $\mu=5\%$ and $\sigma=0.20$.

(a) Determine the price $F_0$ of this option at time $t=0$ (today).  
(b) Using the formulas provided in the lecturs, calculate the value of each of the greeks for this option.  Namely, calculate $\Delta$, $\Gamma$, $\nu$, $\Theta$, and $\rho$.  
(c) Find a formula for the change of the option price with respect to a change in the strike price. In other words, determine $\frac{\delta f(S_t,t)}{\delta K}$  
(d) For each of the suggested modifications below, use an approximation to determine the change in the price of the option above without actually recalculating the price. For each provide an intuitive argument to explain why the price increases or decreases.  
    (i) The rate of return $\mu$ increases to $6\%$  
    (ii) The interest rate $r$ increases to $3\%$  
    (iii) The volatility increases to $\sigma=0.3$  
    (iv) The expiration date changes to 4 months  
    (v) The strike price goes up to $\$11$  
    (vi) The current stock price rises instantaneously to $\$11$

### Solution to Part (a)

### Solution to Part (b)

### Solution to Part (c)

### Solution to Part (d)

## Problem 2 - Estimating parameters and option pricing

This problem is a continuation of Prob. 4 in HW5. We consider the same situation and parameters for your chosen stock.  
**Part 1 - Greeks**. Consider the call option that is ”at-the-money”. Calculate the value of Delta for this option using the formulas given in the lectures. Record the stock price one day later than your original time t = 0 and provide an estimate of the new option price on that day using your value of Delta. How does the estimate compare to the new actual market price of the call? How could you have improved your estimate?  
**Part 2 - Volatility Smile**. Record the current price $S_0$ of your chosen stock, as well as the price of several (about 10) call options with a common expiration date but different strike prices. For each option, use the current market price $C_0$, the expiration $T$ and the strike price $K$ in Black-Scholes formula and solve for the volatility $\sigma$ (see below for details). Plot the volatility as a function of the strike price. Comment on the result and what you can conclude from your graph.

*Solving for* $\sigma$.  There is no explicit way to solve for $\sigma$ in the Black-Scholes formula. You have to approach this numerically. Here are some possible approaches  
* If you are familiar with numerical equation solvers in your language of choice, you can use one of these to solve for your parameters. However, please double-check your results as pre-coded solvers do not always behave in the way that you expect and if you didn’t code it yourself, it’s hard to know exactly what it does.  
* If you are familiar with numerical methods (e.g. Newton’s method) to solve a nonlinear equation, you can use that and code it yourself.  
* Finally, a fairly direct method is to proceed empirically: you calculate the value in the Black-Scholes formula for a braid range of volatilities (e.g (0.1, 0.2, 0.3, · · · , 9.9)) and you consider the value of σ that yields the closest value to your objective.


### Part 1

In [5]:
'''Code Block'''
etf, etf_data = stocal.asset_data(ticker='LQD',start_date='2022-03-29',end_date='2022-03-31')
expiry, years_to_expiry = stocal.maturity(0.25, etf)
px_last = etf_data.iloc[-1,3]
options = etf.option_chain(expiry)
calls = options.calls

[*********************100%***********************]  1 of 1 completed


In [6]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,LQD220715C00120000,2022-03-11 16:35:58+00:00,120.0,2.94,2.98,3.15,0.00,0.00000,19,19,0.097665,True,REGULAR,USD
1,LQD220715C00121000,2022-03-18 14:04:10+00:00,121.0,2.60,2.42,2.59,0.00,0.00000,4,154,0.096567,True,REGULAR,USD
2,LQD220715C00123000,2022-03-30 18:30:52+00:00,123.0,1.50,1.50,1.65,0.00,0.00000,150,157,0.093881,False,REGULAR,USD
3,LQD220715C00124000,2022-03-29 19:24:28+00:00,124.0,1.10,1.03,1.29,0.00,0.00000,4,33,0.093149,False,REGULAR,USD
4,LQD220715C00125000,2022-03-22 18:58:38+00:00,125.0,0.86,0.86,1.00,0.00,0.00000,1,20,0.092966,False,REGULAR,USD
5,LQD220715C00126000,2022-02-23 17:56:39+00:00,126.0,1.00,0.47,0.71,0.00,0.00000,5,99,0.090036,False,REGULAR,USD
6,LQD220715C00127000,2022-02-11 17:11:15+00:00,127.0,1.26,0.23,1.02,0.00,0.00000,66,143,0.116464,False,REGULAR,USD
7,LQD220715C00128000,2022-02-25 14:40:31+00:00,128.0,0.51,0.25,0.46,0.00,0.00000,5,247,0.094980,False,REGULAR,USD
8,LQD220715C00129000,2022-02-09 16:59:03+00:00,129.0,1.12,0.07,0.66,0.00,0.00000,66,337,0.116830,False,REGULAR,USD
9,LQD220715C00130000,2022-04-01 16:17:12+00:00,130.0,0.21,0.18,0.28,-0.64,-75.29412,4,61,0.097909,False,REGULAR,USD


### Part 2

In [3]:
'''Code Block'''

'Code Block'

## Problem 3 - Solving the Black-Scholes PDE

The purpose of this problem is to solve the Black-Scholes PDE with analytical techniques, which will lead us back to the Black-Scholes formula. The technique is very similar to the one used with the Feynman-Kac formula in Prob. 3 of HW8 in MATH 467.

Let's consider the PDE given by
$$\frac{\delta f}{\delta t} + \frac{1}{2}\sigma^2x^2\frac{\delta^2 f}{\delta x^2} + rx\frac{\delta f}{\delta x} - rf = 0$$
with terminal condition $f(T,x)=(x-K)_+$.  The solution $f(t,x)$ corresponds tot he price of a call option (given the initial condition) at time $t$ if the stock price is $x$.  
(a) The first two things that prevent us from solving this PDE directly is (i) the fact that we have a terminal condition, instead of an initial condition; and (ii) that the terms infront of the derivatives are not constant.  To address these, we use the transformation $g(t,x)=f(T-t,e^x)$, equivalent to $f(t,x)=g(T-t,log(x))$.  Under this condition, determine the PDE satisfied by $g$.  
(b) Now, the PDE has an initial condition and constant coefficient, but it still includes a first order derivative in $x$ and the function $g$ itself.  In order to simplify this, use the transformation $h(t,x)=e^{ax+bt}g(t,x)$ and determine the appropriate values of $a$ and $b$ in order for $h$ to solve the heat equation (see (c)).  
(c) We know that the solution to the heat equation $\frac{\delta u}{\delta t} - \frac{\alpha}{2}\frac{\delta^2 u}{\delta x^2} = 0$ with initial condition $u_0$ is given by
$$u(t,x) = \int_{\mathbb{R}} \frac{1}{\sqrt{2\pi\alpha t}}\mathrm{exp}\{-\frac{(x-y)^2}{2\alpha t}\}u_0(y)dy$$
Use this result to determine the function $h$.  
(d) Work the transformations backwards to determine the function $g$, and then $f$.  You should recover the Black-Scholes formula.

### Solution to Part (a)

### Solution to Part (b)

### Solution to Part (c)

### Solution to Part (d)